# Tâche #1 : Classification d'incidents avec un réseau *feedforward* et des *embeddings* Spacy

On reprend la classification des descriptions d’accidents du premier travail. Le corpus de textes contient 3 partitions : 
-	un fichier d’entraînement -  data/incidents_train.json
-	un fichier de validation -  data/incidents_dev.json
-	un fichier de test - data/incidents_test.json

Entraînez un modèle de réseau de neurones de type feedforward multicouche (MLP) avec plongements de mots pour déterminer le type d’un incident à partir de sa description. 

Voici les consignes pour cette tâche : 

-	Nom du notebook : mlp.ipynb
-	Tokenisation : Utilisation de Spacy. 
-	Plongements de mots : Ceux de Spacy. 
-	Normalisation : Aucune normalisation. 
-	Agrégation des plongements de mots : Comparer les approches max, average et min pooling. 
-	Structure du réseau : 1 seule couche cachée dont vous choisirez la taille (à expliquer). 
-	Présentez clairement vos résultats et faites-en l’analyse. En cas de doute, inspirez-vous de ce qui a été fait dans le travail pratique #1. 

Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle. 

## 1. Création du jeu de données (*dataset*)

In [15]:
import json
import spacy
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader



In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [17]:
# Charger le modèle de langue de spaCy 
# !python -m spacy download fr_core_news_md pour télécharger
nlp = spacy.load('fr_core_news_md')

# Définir le Dataset
class IncidentDataset(Dataset):
    def __init__(self, file):
        with open(file, 'r') as f:
            self.data = json.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]['text']
        label = int(self.data[idx]['label'])
        if label > 0:
            label -= 1  # décaler les étiquettes
        # Tokenisation et plongements de mots avec spaCy
        doc = nlp(text)
        embeddings = [token.vector for token in doc]
        return torch.tensor(embeddings), label


In [18]:
# print current directory
import os
print(os.getcwd())

c:\Users\taha\Coding\Academic-Projects\Artificial Neural Networks\Laval


In [19]:

# Charger les données
train_data = IncidentDataset(r'data\incidents_train.json')
dev_data = IncidentDataset(r'data\incidents_dev.json')
test_data = IncidentDataset(r'data\incidents_test.json')


## 2. Gestion de plongements de mots (*embeddings*)

## 3. Création de modèle(s)

In [20]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, pooling='max'):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.pooling = pooling

    def forward(self, x):
        if self.pooling == 'max':
            x = torch.max(x, dim=1)[0]  # max pooling
        elif self.pooling == 'mean':
            x = torch.mean(x, dim=1)  # mean pooling
        elif self.pooling == 'min':
            x = torch.min(x, dim=1)[0]  # min pooling
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

In [21]:
# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()


In [22]:
from torch.nn.utils.rnn import pad_sequence
# Dans votre DataLoader
def collate_fn(batch):
    inputs = pad_sequence([item[0] for item in batch], batch_first=True)
    labels = torch.tensor([item[1] for item in batch])
    return inputs, labels

## 5. Entraînement de modèle(s)

In [23]:
# Initialiser le MLP avec différentes tailles de couche cachée et méthodes d'agrégation
n_classes = len(set([label for _, label in train_data]))  # nombre d'étiquettes uniques dans les données d'entraînement
for hidden_dim in [50, 100, 200]:
    for pooling in ['max', 'mean', 'min']:
        model = MLP(300, hidden_dim, n_classes, pooling).to(device)  # 300 = dimension des plongements

        # Définir l'optimiseur
        optimizer = torch.optim.Adam(model.parameters())
        # Définir le DataLoader
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_fn)
        dev_loader = DataLoader(dev_data, batch_size=32, shuffle=False, collate_fn=collate_fn)
        test_loader = DataLoader(test_data, batch_size=32, shuffle=False, collate_fn=collate_fn)

        # Boucle d'entraînement
        for epoch in range(1):  # 10 = nombre d'époques
            for i, (inputs, labels) in enumerate(train_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

KeyboardInterrupt: 

## 6. Évaluation et analyse de résultats

In [ ]:
# Évaluation du modèle sur les données de test
from sklearn.metrics import classification_report
model.eval()
predictions, true_labels = [], []
for inputs, labels in test_loader:
    inputs = inputs.to(device)
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    predictions.extend(predicted.tolist())
    true_labels.extend(labels.tolist())

# Afficher le rapport de classification
print(classification_report(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.33      0.01      0.02       110
           1       0.48      0.34      0.40        58
           2       1.00      0.07      0.12        15
           3       0.00      0.00      0.00         2
           4       0.52      0.61      0.56       191
           5       0.07      0.21      0.11        29
           6       0.22      0.18      0.20        66
           7       0.13      0.27      0.17        60

    accuracy                           0.32       531
   macro avg       0.34      0.21      0.20       531
weighted avg       0.38      0.32      0.30       531



c:\Users\taha\Coding\ocr_test\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\taha\Coding\ocr_test\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\taha\Coding\ocr_test\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
